In [ ]:
try:
    # If this can be imported, it assumes you are on the TU Delft network with access to cluster
    import hpc05
    client = hpc05.Client(profile='pbs', timeout=60)
    print("Connected to hpc05")
except ImportError:
    from ipyparallel import Client
    client = Client(profile='python3')
    print("Connected to local engines")

In [ ]:
dview = client[:]
lview = client.load_balanced_view()
print(len(dview))

In [ ]:
%matplotlib inline
import holoviews as hv
hv.notebook_extension('bokeh')
import sympy.interactive
sympy.interactive.init_printing('mathjax')

In [ ]:
# %%px --local

import kwant
import numpy as np
from itertools import product
import os.path
import deepdish as dd
import sys
sys.path.append('/home/bnijholt/Dropbox/Work/nanowire_current/')
sys.path.append('/home/basnijholt/Dropbox/Work/nanowire_current/')
from supercurrent import (constants,
                          I_c,
                          make_params,
                          SimpleNamespace,
                          make_3d_wire,
                          save_data)

# Creating some wires as example

In [ ]:
# Create system with infinite leads
syst_params = dict(a=10, L=30, r1=50, r2=70, phi=185, angle=0, holes=True, spin=True,
                   disorder=False, with_vlead=True, with_leads=True, L_sc=10, with_shell=True,
                   shape='square')
syst, hopping = make_3d_wire(**syst_params)
kwant.plot(syst)

In [ ]:
# Create system without leads
syst_params['with_leads'] = False
syst, hopping = make_3d_wire(**syst_params)
kwant.plot(syst)

In [ ]:
# Create system without leads and bigger SC regions
syst_params['L_sc'] = 30
syst, hopping = make_3d_wire(**syst_params)
kwant.plot(syst)

# Mean free path

In [ ]:
%%px --local
def transmission(syst, p):
    smatrix = kwant.smatrix(syst, args=[p])
    return smatrix.transmission(0, 1), smatrix.num_propagating(0)

p = make_params(Delta=0, alpha=20, mu=50, B_x=0, B_y=0, B_z=0, disorder=None, salt=None)
syst_params = dict(a=8, L=None, r1=50, r2=70, phi=135, angle=0, disorder=True, with_vlead=False,
                   with_leads=True, L_sc=8, with_shell=False, holes=False, spin=True, shape='circle')

In [ ]:
Ls = np.arange(80, 500, 80)
conductances = []
num_propagating = []
for L in Ls:
    print(L)
    dview.execute('syst_params["L"] = {}'.format(L))
    %px syst, hopping = make_3d_wire(**syst_params)
    salts = range(30)
    disorders = np.arange(0, 150, 10)
    conductance = lview.map_async(lambda x: transmission(syst, p.update(salt=x[0], disorder=x[1])), 
                        product(salts, disorders))
    conductance.wait_interactive()
    conductance = np.array(conductance.result()).reshape(len(salts), len(disorders), -1)
    num_propagating.append(conductance[:, :, 1])
    conductances.append(conductance[:, :, 0])

In [ ]:
save_data('mfp__mu_{:.0f}_meV__a_{:.0f}_nm.h5'.format(p.mu, syst_params['a']), p, constants, None, None, syst_params, 
          T=None, current_phase=None, conductances=conductances, salts=salts,
          disorders=disorders, Ls=Ls, num_propagating=num_propagating)

# No disorder $B_x$ vs $I_c$:

In [ ]:
%%px --local 

syst_params = dict(a=8, r1=50, r2=70, phi=135, angle=0,
                   disorder=False, with_vlead=True,
                   with_shell=True, holes=True, spin=True,
                   L=None, with_leads=None, L_sc=None, shape='circle')

hamiltonian_params = dict(Delta=60, t_interface=7/8*constants.t, 
                          mu=None, B_x=None, alpha=None, g=None, orbital=None)
tol = 1e-2
Bs = np.linspace(0, 1, 100)
max_frequencies = 200

In [ ]:
T_list = [0.1, 0.5, 1]
orbital_list = [False, True]
g_list = [0, 50]
alpha_list = [0, 20]
mu_list = [10, 20, 30, 40, 50, 60]

for L in [80, 160, 320, 640]:
    for with_leads, L_sc in [(True, 8), (False, 400)]:
        syst_params['L'] = L
        syst_params['with_leads'] = with_leads
        syst_params['L_sc'] = L_sc
        fname = 'no_disorder_B_vs_I_c__L_{}_nm{}.h5'.format(L, '__infinite_leads' if with_leads else '')
        if not os.path.isfile(fname):
            print(fname)
            dview['syst_params'] = syst_params
            %px p = make_params(**hamiltonian_params)
            %px syst, hopping = make_3d_wire(**syst_params)
            vals = list(product(T_list, Bs, orbital_list, g_list, alpha_list, mu_list))
            I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_x=x[1], orbital=x[2], g=x[3], alpha=x[4], mu=x[5]),
                                              x[0], tol, max_frequencies), vals)
            I.wait_interactive()
            current_phase = I.result()
            save_data(fname, make_params(**hamiltonian_params), constants,
                      Bs, tol, syst_params, None, current_phase, 
                      max_frequencies=max_frequencies, vals=vals, 
                      T_list=T_list, orbital_list=orbital_list, 
                      g_list=g_list, alpha_list=alpha_list, mu_list=mu_list)

# $I_c(B_x, \mu)$

In [ ]:
%%px --local 

syst_params = dict(a=8, r1=50, r2=70, phi=135, angle=0,
                   disorder=False, with_vlead=True,
                   with_shell=True, holes=True, spin=True,
                   L=640, with_leads=True, L_sc=8, shape='circle')

hamiltonian_params = dict(Delta=60, t_interface=7/8*constants.t,
                          mu=None, B_x=None, alpha=None, g=None, orbital=None)
tol = 1e-2
Bs = np.linspace(0, 1, 100)
max_frequencies = 200
T = 1
syst, hopping = make_3d_wire(**syst_params)
p = make_params(**hamiltonian_params)

In [ ]:
orbital_list = [False, True]
g_list = [0, 50]
alpha_list = [0, 20]
mu_list = np.linspace(0, 50, 100)

fname = 'no_disorder_B_vs_I_c__L_{}_nm{}.h5'.format(syst_params['L'], 
            '__infinite_leads' if syst_params['with_leads'] else '')
if not os.path.isfile(fname):
    vals = list(product(Bs, orbital_list, g_list, alpha_list, mu_list))
    I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_x=x[0], orbital=x[1], g=x[2], alpha=x[3], mu=x[4]),
                                      T, tol, max_frequencies), vals)
    I.wait_interactive()
    current_phase = I.result()
    save_data(fname, make_params(**hamiltonian_params), constants,
              Bs, tol, syst_params, T, current_phase, 
              max_frequencies=max_frequencies, vals=vals, 
              orbital_list=orbital_list, 
              g_list=g_list, alpha_list=alpha_list, mu_list=mu_list)

# Field in y

In [ ]:
%%px --local 

syst_params = dict(a=8, r1=50, r2=70, phi=135, angle=0,
                   disorder=False, with_vlead=True,
                   with_shell=True, holes=True, spin=True,
                   L=640, with_leads=True, L_sc=8, shape='circle')

hamiltonian_params = dict(Delta=60, t_interface=7/8*constants.t,
                          mu=None, B_y=None, alpha=None, g=None, orbital=None)
tol = 1e-2
Bs = np.linspace(0, 1, 100)
max_frequencies = 200
T = 1
syst, hopping = make_3d_wire(**syst_params)
p = make_params(**hamiltonian_params)

In [ ]:
orbital_list = [False, True]
g_list = [0, 50]
alpha_list = [0, 20]
mu_list = [10, 50]

fname = 'no_disorder_B_y_vs_I_c__L_{}_nm{}.h5'.format(syst_params['L'], 
                                                      '__infinite_leads' if syst_params['with_leads'] else '')
if not os.path.isfile(fname):
    vals = list(product(Bs, orbital_list, g_list, alpha_list, mu_list))
    I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_y=x[0], orbital=x[1], g=x[2], alpha=x[3], mu=x[4]),
                                      T, tol, max_frequencies), vals)
    I.wait_interactive()
    current_phase = I.result()
    save_data(fname, make_params(**hamiltonian_params), constants,
              Bs, tol, syst_params, T, current_phase, 
              max_frequencies=max_frequencies, vals=vals, 
              orbital_list=orbital_list, 
              g_list=g_list, alpha_list=alpha_list, mu_list=mu_list,
              Bys=Bs)

# Rotation of the field

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/558px-3D_Spherical.svg.png)

In [ ]:
%%px --local 

syst_params = dict(a=8, r1=50, r2=70, phi=135, angle=0,
                   disorder=False, with_vlead=True,
                   with_shell=True, holes=True, spin=True,
                   L=640, with_leads=True, L_sc=8, shape='circle')

hamiltonian_params = dict(Delta=60, t_interface=7/8*constants.t,
                          mu=50, B_x=None, B_z=None, alpha=20, g=50, orbital=True)
tol = 1e-2
max_frequencies = 200

def polar_to_z(r, theta):
    return r * np.exp(1j * theta)

Bs = np.linspace(0, 1, 100)
thetas = np.linspace(0, 2 * np.pi, 12, endpoint=False)
xy = polar_to_z(Bs, thetas[:, None])
B_xs = xy.real
B_zs = xy.imag
T = 1

In [ ]:
fname = 'rotation_of_field__L_{}_nm{}.h5'.format(
    syst_params['L'], '__infinite_leads' if syst_params['with_leads'] else '')
if not os.path.isfile(fname):
    print(fname)
    %px p = make_params(**hamiltonian_params)
    %px syst, hopping = make_3d_wire(**syst_params)
    vals = list(zip(B_xs.reshape(-1), B_zs.reshape(-1)))
    I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_x=x[0], B_z=x[1]),
                                      T, tol, max_frequencies), vals)
    I.wait_interactive()
    current_phase = I.result()
    save_data(fname, make_params(**hamiltonian_params), constants,
              None, tol, syst_params, T, current_phase,
              max_frequencies=max_frequencies, vals=vals, theta=thetas.repeat(len(Bs)),
              B=np.linalg.norm(vals, axis=1))

# Induced gap

In [ ]:
%%px --local
# Create system with infinite leads
syst_params = dict(a=8, L=800, r1=50, r2=70, phi=135, angle=0, 
                   disorder=False, with_vlead=True, with_leads=True, L_sc=400,
                   with_shell=True, spin=True, holes=True)
syst, hopping = make_3d_wire(**syst_params)
lead = syst.leads[1]

p = make_params(t_interface=7/8*constants.t)

def evs(lead, p):
    h0 = lead.cell_hamiltonian(args=[p])
    t0 = lead.inter_cell_hopping(args=[p])
    ham = h0 + t0 + t0.conj().T
    ev = np.linalg.eigvalsh(ham)
    return np.abs(ev).min()

In [ ]:
mus = np.linspace(2, 20, 50)
deltas = np.linspace(0, 100, 50)
vals = list(product(deltas, mus))
dview.scatter('vals', vals, block=True)
%px res = [evs(lead, p) for p.Delta, p.mu in vals]
res = np.reshape(dview.gather('res', block=True), (len(deltas), len(mus)))
print("Max gap is {} meV".format(max(res.min(axis=1))))

In [ ]:
hv.Curve((deltas, res.min(axis=1)), kdims=['$\Delta$'], vdims=['$E_{gap}$'], label="$E_{gap}$ over range of $\Delta$'s")

In [ ]:
Delta_ind = {delta: hv.Path((mus, evs), kdims=['$\mu$', ('E_gap', '$E_{gap}$')]) 
      for evs, delta in zip(res, deltas)}

hm = hv.HoloMap(Delta_ind, kdims=['$\Delta$'])
(hm.select(E_gap=(0.2, 0.3)) * hv.HLine(0.25))

# Gate

In [ ]:
# Plot test
syst_params = dict(a=8, L=160, r1=50, r2=70, phi=135, angle=0, holes=True, spin=True,
                   disorder=False, with_vlead=True, with_leads=True, L_sc=8, with_shell=True,
                   shape='circle')
syst, hopping = make_3d_wire(**syst_params)
gate_fun = gate(syst, 100, 160)
sites = [gate_fun(pos[0]) for pos in [i.pos for i in syst.sites]]
kwant.plot(syst, site_lw=0, site_color=sites, colorbar=False)

In [ ]:
%%px --local 

def find_nearest(array, value):
    idx = np.abs(np.array(array) - value).argmin()
    return array[idx]

def gate(syst, V, gate_size):
    x_positions = sorted(set(i.pos[0] for i in syst.sites))
    x_mid = (max(x_positions) - min(x_positions)) / 2
    x_L = find_nearest(x_positions, x_mid - gate_size / 2)
    x_R = find_nearest(x_positions, x_mid + gate_size / 2)
    return lambda x: V if x > x_L and x <= x_R else 0

syst_params = dict(a=8, r1=50, r2=70, phi=135, angle=0,
                   disorder=False, with_vlead=True,
                   with_shell=True, holes=True, spin=True,
                   L=640, with_leads=True, L_sc=8,
                   shape='circle')

hamiltonian_params = dict(Delta=60, t_interface=7/8*constants.t, 
                          mu=50, B_x=None, alpha=20, g=50, orbital=True)
tol = 1e-2
Bs = np.linspace(0, 1, 100)
Vs = np.linspace(0, 40, 100)
max_frequencies = 200
T = 1
gate_size = None

p = make_params(**hamiltonian_params)
syst, hopping = make_3d_wire(**syst_params)

In [ ]:
vals = list(product(Bs, Vs))

for gate_size in [640]:#[80, 160, 320, 640]:
    dview['gate_size'] = gate_size
    I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_x=x[0], V=gate(syst, x[1], gate_size)),
                                      T, tol, max_frequencies), vals)
    I.wait_interactive()
    current_phase = I.result()
    fname = "current_as_function_of_gate_and_B_x__gate_size_{}_nm.h5".format(gate_size)
    save_data(fname, make_params(**hamiltonian_params), constants,
              Bs, tol, syst_params, T, current_phase, 
              max_frequencies=max_frequencies, vals=vals, Vs=Vs, gate_size=gate_size)